In [1]:
import os
import glob
import pandas as pd
import numpy as np
import constants as const
from tqdm import tqdm
from scipy.fft import fft, fftshift

In [2]:
def preprocess(src_folder=const.WINDOWS_FOLDER, dst_folder=const.PREPROCESSED_FOLDER):
    os.makedirs(dst_folder, exist_ok=True)
    
    for obj in tqdm(os.listdir(src_folder)):
        obj_path = os.path.join(src_folder, obj)

        rows = []
        for window in os.listdir(obj_path):
            if const.CSV not in window:
                continue
            window_path = os.path.join(obj_path, window)
            df = pd.read_csv(window_path)
            label = df[const.LABEL].mode()[0]
            row = {const.LABEL: label}
            for col in const.COLUMNS:
                data = df[col].values
                num_samples = len(data)
                yf = fft(data)[:num_samples//2]
                yf = 1.0/num_samples * np.abs(yf)
                row.update({f"{col}_{i:02d}": v for i, v in enumerate(yf)})
            rows.append(row)
        
        dst_path = f"{dst_folder}/{obj}.csv"
        pd.DataFrame(rows).to_csv(dst_path, index=False)

In [3]:
preprocess()

100%|██████████| 490/490 [01:53<00:00,  4.33it/s]
